In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import requests
import json
import datetime
import time

# 미세먼지 데이터 처리

In [2]:
#  온도/기온/풍속/습도/기압/일조/일사
file_list = os.listdir('./data/origin_data_file/weather')

pd_weather = []
for file_name in file_list:
    if file_name == '.ipynb_checkpoints':
        continue
    tmp = pd.read_csv(f'./data/origin_data_file/weather/{file_name}',encoding='cp949').drop(columns=['지점'])
    tmp['일시'] = pd.to_datetime(tmp['일시'])
    pd_weather.append(tmp)
pd_weather = pd.concat(pd_weather,axis=0).sort_values(by=['일시','지점명'])
pd_weather = pd_weather[pd_weather['지점명'] == '서울'].reset_index(drop=True)
pd_weather 

,지점명,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),현지기압(hPa),일조(hr),일사(MJ/m2),전운량(10분위),중하층운량(10분위),최저운고(100m ),시정(10m)
0,서울,2016-01-01 00:00:00,-1.9,NaN,0.1,0.0,85.0,1022.1,NaN,NaN,0.0,0.0,NaN,500.0
1,서울,2016-01-01 01:00:00,-2.1,NaN,1.6,90.0,83.0,1022.0,NaN,NaN,NaN,NaN,NaN,600.0
2,서울,2016-01-01 02:00:00,-2.2,NaN,0.4,0.0,86.0,1022.2,NaN,NaN,NaN,NaN,NaN,500.0
3,서울,2016-01-01 03:00:00,-2.5,NaN,1.8,90.0,90.0,1022.2,NaN,NaN,0.0,0.0,NaN,400.0
4,서울,2016-01-01 04:00:00,-2.9,NaN,1.9,70.0,90.0,1022.0,NaN,NaN,0.0,0.0,NaN,400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52603,서울,2021-12-31 19:00:00,-6.4,NaN,2.2,250.0,34.0,1021.1,NaN,NaN,0.0,0.0,NaN,2000.0
52604,서울,2021-12-31 20:00:00,-6.3,NaN,4.1,320.0,35.0,1021.2,NaN,NaN,0.0,0.0,NaN,2000.0
52605,서울,2021-12-31 21:00:00,-6.7,NaN,4.8,320.0,36.0,1021.2,NaN,NaN,0.0,0.0,NaN,2000.0
52606,서울,2021-12-31 22:00:00,-7.5,NaN,3.0,320.0,37.0,1021.8,NaN,NaN,0.0,0.0,NaN,2000.0


In [3]:
#  미세먼지
file_list = os.listdir('./data/origin_data_file/pm10')

pd_pm10 = []
for file_name in file_list:
    if file_name == '.ipynb_checkpoints':
        continue
    tmp = pd.read_csv(f'./data/origin_data_file/pm10/{file_name}',encoding='cp949').drop(columns=['지점'])
    tmp['일시'] = pd.to_datetime(tmp['일시'])
    pd_pm10.append(tmp)
pd_pm10 = pd.concat(pd_pm10,axis=0).sort_values(by=['일시','지점명']).reset_index(drop=True)
pd_pm10

,지점명,일시,1시간평균 미세먼지농도(㎍/㎥)
0,관악산,2016-01-01 00:00:00,26
1,서울,2016-01-01 00:00:00,73
2,수원,2016-01-01 00:00:00,70
3,관악산,2016-01-01 01:00:00,36
4,서울,2016-01-01 01:00:00,72
...,...,...,...
144083,서울,2021-12-31 22:00:00,26
144084,수원,2021-12-31 22:00:00,33
144085,관악산,2021-12-31 23:00:00,17
144086,서울,2021-12-31 23:00:00,24


In [4]:
pd_pm10_pivot = pd_pm10.pivot(index='일시',columns='지점명',values='1시간평균 미세먼지농도(㎍/㎥)').copy()
pd_pm10_pivot

지점명,관악산,서울,수원
일시,,,
2016-01-01 00:00:00,26.0,73.0,70.0
2016-01-01 01:00:00,36.0,72.0,71.0
2016-01-01 02:00:00,44.0,78.0,84.0
2016-01-01 04:00:00,53.0,89.0,103.0
2016-01-01 05:00:00,49.0,85.0,96.0
...,...,...,...
2021-12-31 19:00:00,35.0,22.0,28.0
2021-12-31 20:00:00,27.0,27.0,34.0
2021-12-31 21:00:00,37.0,23.0,25.0


In [5]:
pd_pm10.isna().sum()

지점명                  0
일시                   0
1시간평균 미세먼지농도(㎍/㎥)    0
dtype: int64

In [6]:
pd_weather['강수량(mm)'] = pd_weather['강수량(mm)'].fillna(value=0.0)
pd_weather['일조(hr)'] = pd_weather['일조(hr)'].fillna(value=0.0)
pd_weather['일사(MJ/m2)'] = pd_weather['일사(MJ/m2)'].fillna(value=0.0)
pd_weather['시정(10m)'] = pd_weather['시정(10m)'].interpolate()
pd_weather['습도(%)'] = pd_weather['습도(%)'].interpolate()
pd_weather['풍속(m/s)'] = pd_weather['풍속(m/s)'].interpolate()
pd_weather['풍향(16방위)'] = pd_weather['풍향(16방위)'].fillna(method='ffill')
pd_weather['현지기압(hPa)'] = pd_weather['현지기압(hPa)'].interpolate()
pd_weather.index = pd_weather['일시']
pd_weather.drop(columns=['일시','지점명','전운량(10분위)','중하층운량(10분위)','최저운고(100m )'],inplace=True)
pd_weather

,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),현지기압(hPa),일조(hr),일사(MJ/m2),시정(10m)
일시,,,,,,,,,
2016-01-01 00:00:00,-1.9,0.0,0.1,0.0,85.0,1022.1,0.0,0.0,500.0
2016-01-01 01:00:00,-2.1,0.0,1.6,90.0,83.0,1022.0,0.0,0.0,600.0
2016-01-01 02:00:00,-2.2,0.0,0.4,0.0,86.0,1022.2,0.0,0.0,500.0
2016-01-01 03:00:00,-2.5,0.0,1.8,90.0,90.0,1022.2,0.0,0.0,400.0
2016-01-01 04:00:00,-2.9,0.0,1.9,70.0,90.0,1022.0,0.0,0.0,400.0
...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00,-6.4,0.0,2.2,250.0,34.0,1021.1,0.0,0.0,2000.0
2021-12-31 20:00:00,-6.3,0.0,4.1,320.0,35.0,1021.2,0.0,0.0,2000.0
2021-12-31 21:00:00,-6.7,0.0,4.8,320.0,36.0,1021.2,0.0,0.0,2000.0


In [7]:
pd_weather.isna().sum()

기온(°C)       6
강수량(mm)      0
풍속(m/s)      0
풍향(16방위)     0
습도(%)        0
현지기압(hPa)    0
일조(hr)       0
일사(MJ/m2)    0
시정(10m)      0
dtype: int64

In [8]:
total_wheather = pd.concat([pd_weather,pd_pm10_pivot],axis=1)
total_wheather['관악산'].interpolate(inplace=True)
total_wheather['서울'].interpolate(inplace=True)
total_wheather['수원'].interpolate(inplace=True)
total_wheather.columns = ['기온','강수량','풍속','풍향','습도','기압','일조량','일사량','시정','관악pm10','서울pm10','수원pm10']
total_wheather.to_csv('./data/weather_and_pm10.csv')

In [9]:
#길이 체크
6*365*24+24*2#2016, 2020 -> 윤년

52608

# 코스피 데이터 처리

In [10]:
import FinanceDataReader as fdr
import pandas_datareader as pdr

In [11]:
stocks = fdr.StockListing('KOSPI') # 코스피, 코스닥, 코넥스 전체
stocks

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,59600,2,-400,-0.67,59700,60000,59300,13175845,785445073900,355799039980000,5969782550,STK
1,373220,KR7373220003,LG에너지솔루션,KOSPI,,569000,1,6000,1.07,564000,585000,560000,586500,335459241000,133146000000000,234000000,STK
2,207940,KR7207940008,삼성바이오로직스,KOSPI,,873000,2,-9000,-1.02,876000,879000,867000,69014,60138020000,62134902000000,71174000,STK
3,000660,KR7000660001,SK하이닉스,KOSPI,,84500,1,600,0.72,84200,85500,83600,2530308,214496226084,61516199842500,728002365,STK
4,006400,KR7006400006,삼성SDI,KOSPI,,725000,2,-35000,-4.61,750000,754000,715000,492155,360219221000,49854284250000,68764530,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,002785,KR7002781003,진흥기업우B,KOSPI,,4945,1,15,0.30,4950,4950,4885,50,247195,4238943010,857218,STK
936,001527,KR7001522002,동양2우B,KOSPI,,13100,1,250,1.95,13150,13300,13100,1232,16296100,4042253900,308569,STK
937,001525,KR7001521004,동양우,KOSPI,,6330,1,30,0.48,6300,6340,6300,281,1776130,3910237230,617731,STK
938,002787,KR7002782001,진흥기업2우B,KOSPI,,13050,2,-50,-0.38,12850,13050,12850,38,489050,3847244400,294808,STK


In [12]:
KOSPI_VOL150_LIST = stocks.sort_values(by='Volume',ascending=False).head(200).copy().reset_index(drop=True)
KOSPI_VOL150_LIST

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,096300,KR7096300009,베트남개발1,KOSPI,,214,2,-11,-4.89,226,234,209,44880541,9812038289,29463565154,137680211,STK
1,001440,KR7001440007,대한전선,KOSPI,,1685,2,-35,-2.03,1745,1760,1670,40696613,69925464990,2096937020165,1244473009,STK
2,323410,KR7323410001,카카오뱅크,KOSPI,,20250,1,2950,17.05,17650,20500,17650,25098201,490827953350,9651837224250,476633937,STK
3,034020,KR7034020008,두산에너빌리티,KOSPI,,14400,2,-1450,-9.15,15400,15650,14200,20180028,297692900000,9191635675200,638308033,STK
4,001570,KR7001570001,금양,KOSPI,,31850,1,2950,10.21,29250,34550,28900,15721019,497728104450,1848893678450,58050037,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,003410,KR7003410008,쌍용C&E,KOSPI,,5770,2,-30,-0.52,5740,5810,5720,313111,1806923770,2907269863150,503859595,STK
196,001390,KR7001390004,KG케미칼,KOSPI,,23800,1,750,3.25,23250,24650,22600,309145,7367945250,330013084800,13866096,STK
197,081660,KR7081660003,휠라홀딩스,KOSPI,,33650,1,400,1.20,33050,33900,32750,301919,10114903150,2044312270300,60752222,STK
198,002790,KR7002790004,아모레G,KOSPI,,25950,1,400,1.57,25300,26150,24550,300938,7654038700,2139789771000,82458180,STK


In [13]:
start_date = '2016-01-01'
end_date = '2022-01-01'

stock_data = []
col_name = []
for i in range(200):
    Code = KOSPI_VOL150_LIST['Code'][i]
    Name = KOSPI_VOL150_LIST['Name'][i]
    
    tmp = fdr.DataReader(Code,start=start_date,end=end_date).Close
    
    if tmp.shape[0] < 0.85*(52*5*6):#(52주 * 5영업일 * 6년)
        print(f'{Name:<15} : {tmp.shape[0]}')
        continue
    
    col_name.append(f'{Name}_close')
    stock_data.append(tmp)
stock_data = pd.concat(stock_data,axis=1)
stock_data.columns = col_name
stock_data

카카오뱅크           : 99
우리금융지주          : 715
카카오페이           : 42
SK아이이테크놀로지      : 161
SK바이오사이언스       : 198
한화시스템           : 529
한컴라이프케어         : 93
제이알글로벌리츠        : 346
LG에너지솔루션        : 0
일동제약            : 1312
SK스퀘어           : 24
ESR켄달스퀘어리츠      : 253
NH올원리츠          : 31
현대일렉트릭          : 1144
롯데리츠            : 539
센트랄모텍           : 521
프레스티지바이오파마      : 224
크래프톤            : 97
대덕전자            : 402
넷마블             : 1142
에스디바이오센서        : 114
신한알파리츠          : 838
에어부산            : 744


,베트남개발1_close,대한전선_close,두산에너빌리티_close,금양_close,세원이앤씨_close,삼성전자_close,도화엔지니어링_close,화천기계_close,에이프로젠_close,한미글로벌_close,...,인바이오젠_close,한화_close,SIMPAC_close,보령_close,무림P&P_close,쌍용C&E_close,KG케미칼_close,휠라홀딩스_close,아모레G_close,신송홀딩스_close
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,100,2103,14766,1855,1660,24100,4295,2585,2970,11850,...,1540,38230,5460,8072,4775,3446,15150,19000,146662,9060
2016-01-05,104,2141,14689,1895,1645,24160,4320,2600,2945,11750,...,1575,37938,5250,8145,4810,3533,14950,19180,148657,9330
2016-01-06,102,2103,14421,1910,1753,23500,4235,2590,2950,11700,...,1540,37890,5220,8562,4895,3554,15000,19200,150154,9340
2016-01-07,102,2094,14191,1880,1755,23260,4225,2590,2825,11850,...,1460,36674,5180,8389,4765,3511,14850,18960,154144,9280
2016-01-08,101,2037,13885,1880,1738,23420,4230,2545,2820,11950,...,1490,36577,5350,8633,4760,3565,14700,19700,156639,9040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-24,50,1665,19545,4700,1110,80500,7660,2475,1525,11900,...,3530,31950,6430,14600,4360,7890,30650,35200,45700,5130
2021-12-27,51,1661,19449,4735,1140,80200,7710,2525,1575,12000,...,3555,31850,6460,14850,4325,7890,30350,35200,45600,5220
2021-12-28,51,1675,20028,4750,1135,80300,7790,2500,1530,12050,...,3550,32200,6510,14650,4325,7890,30700,35000,45950,5270


In [14]:
stock_data.isna().sum()

베트남개발1_close     0
대한전선_close       0
두산에너빌리티_close    0
금양_close         0
세원이앤씨_close      0
                ..
쌍용C&E_close      0
KG케미칼_close      0
휠라홀딩스_close      0
아모레G_close       0
신송홀딩스_close      0
Length: 177, dtype: int64

In [15]:
stock_data = stock_data.fillna(method='ffill')
stock_data = stock_data.fillna(value=0.0)
stock_data

,베트남개발1_close,대한전선_close,두산에너빌리티_close,금양_close,세원이앤씨_close,삼성전자_close,도화엔지니어링_close,화천기계_close,에이프로젠_close,한미글로벌_close,...,인바이오젠_close,한화_close,SIMPAC_close,보령_close,무림P&P_close,쌍용C&E_close,KG케미칼_close,휠라홀딩스_close,아모레G_close,신송홀딩스_close
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,100,2103,14766,1855,1660,24100,4295,2585,2970,11850,...,1540,38230,5460,8072,4775,3446,15150,19000,146662,9060
2016-01-05,104,2141,14689,1895,1645,24160,4320,2600,2945,11750,...,1575,37938,5250,8145,4810,3533,14950,19180,148657,9330
2016-01-06,102,2103,14421,1910,1753,23500,4235,2590,2950,11700,...,1540,37890,5220,8562,4895,3554,15000,19200,150154,9340
2016-01-07,102,2094,14191,1880,1755,23260,4225,2590,2825,11850,...,1460,36674,5180,8389,4765,3511,14850,18960,154144,9280
2016-01-08,101,2037,13885,1880,1738,23420,4230,2545,2820,11950,...,1490,36577,5350,8633,4760,3565,14700,19700,156639,9040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-24,50,1665,19545,4700,1110,80500,7660,2475,1525,11900,...,3530,31950,6430,14600,4360,7890,30650,35200,45700,5130
2021-12-27,51,1661,19449,4735,1140,80200,7710,2525,1575,12000,...,3555,31850,6460,14850,4325,7890,30350,35200,45600,5220
2021-12-28,51,1675,20028,4750,1135,80300,7790,2500,1530,12050,...,3550,32200,6510,14650,4325,7890,30700,35000,45950,5270


주요 지수 | 코스피지수 | 코스닥지수 | 원-달러 환율 | 원-엔화 환율 | S&P500 | NASDAQ | 미 10년채 금리 |    국제 원유(두바이유 : 국내 유가 영향)
  ---     |    ---     |     ---    |     ---      |      ---     |   ---  |   ---  |     ---        |              ---
__코드__  |    KS11    |    KQ11    |    USD/KRW   |    JPY/KRW   |  US500 |  IXIC  |     DGS10      |           POILDUBUSDM


In [16]:
KOSPI = fdr.DataReader('KS11',start=start_date,end=end_date)['Close']
KOSPI

Date
2016-01-04    1918.760010
2016-01-05    1930.530029
2016-01-06    1925.430054
2016-01-07    1904.329956
2016-01-08    1917.619995
                 ...     
2021-12-24    3012.429932
2021-12-27    2999.550049
2021-12-28    3020.239990
2021-12-29    2993.290039
2021-12-30    2977.649902
Name: Close, Length: 1473, dtype: float64

In [17]:
LIST_1 = ['KQ11','USD/KRW','JPY/KRW','US500','IXIC']

pd_index1 = []
for L in LIST_1:
    tmp = fdr.DataReader(L,start=start_date,end=end_date)['Close']
    pd_index1.append(tmp)
pd_index1 = pd.concat(pd_index1,axis=1)
pd_index1.columns = LIST_1
pd_index1

,KQ11,USD/KRW,JPY/KRW,US500,IXIC
Date,,,,,
2016-01-01,NaN,1174.109985,9.580900,NaN,NaN
2016-01-04,677.789978,1174.109985,9.585800,2012.660034,4903.089844
2016-01-05,684.070007,1187.359985,9.946800,2016.709961,4891.430176
2016-01-06,687.270020,1188.719971,9.988000,1990.260010,4835.759766
2016-01-07,679.659973,1198.739990,10.111000,1943.089966,4689.430176
...,...,...,...,...,...
2021-12-27,1011.359985,1185.920044,10.363698,4791.189941,15871.259766
2021-12-28,1027.439941,1185.650024,10.334326,4786.350098,15781.719727
2021-12-29,1028.050049,1187.849976,10.349199,4793.060059,15766.219727


In [18]:
A = pdr.DataReader('DGS10','fred',start=start_date, end=end_date)
B = pdr.DataReader('POILDUBUSDM', 'fred', start=start_date, end=end_date)

pd_index2 = pd.concat([A,B],axis=1)
pd_index2 = pd_index2.fillna(method='ffill')
pd_index2

,DGS10,POILDUBUSDM
DATE,,
2016-01-01,NaN,27.458095
2016-01-04,2.24,27.458095
2016-01-05,2.25,27.458095
2016-01-06,2.18,27.458095
2016-01-07,2.16,27.458095
...,...,...
2021-12-28,1.49,72.847826
2021-12-29,1.55,72.847826
2021-12-30,1.52,72.847826


In [19]:
total_stock = pd.merge(left=KOSPI, right=pd_index1, how='left', left_index=True, right_index=True)
total_stock.rename(columns={'Close':"KOSPI"},inplace=True)
total_stock = pd.merge(left=total_stock, right=pd_index2, how='left', left_index=True, right_index=True)
total_stock = pd.merge(left=total_stock, right=stock_data, how='left', left_index=True, right_index=True)
total_stock.fillna(method='ffill',inplace=True)
total_stock

,KOSPI,KQ11,USD/KRW,JPY/KRW,US500,IXIC,DGS10,POILDUBUSDM,베트남개발1_close,대한전선_close,...,인바이오젠_close,한화_close,SIMPAC_close,보령_close,무림P&P_close,쌍용C&E_close,KG케미칼_close,휠라홀딩스_close,아모레G_close,신송홀딩스_close
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,1918.760010,677.789978,1174.109985,9.585800,2012.660034,4903.089844,2.24,27.458095,100,2103,...,1540,38230,5460,8072,4775,3446,15150,19000,146662,9060
2016-01-05,1930.530029,684.070007,1187.359985,9.946800,2016.709961,4891.430176,2.25,27.458095,104,2141,...,1575,37938,5250,8145,4810,3533,14950,19180,148657,9330
2016-01-06,1925.430054,687.270020,1188.719971,9.988000,1990.260010,4835.759766,2.18,27.458095,102,2103,...,1540,37890,5220,8562,4895,3554,15000,19200,150154,9340
2016-01-07,1904.329956,679.659973,1198.739990,10.111000,1943.089966,4689.430176,2.16,27.458095,102,2094,...,1460,36674,5180,8389,4765,3511,14850,18960,154144,9280
2016-01-08,1917.619995,682.559998,1195.390015,10.174000,1922.030029,4643.629883,2.13,27.458095,101,2037,...,1490,36577,5350,8633,4760,3565,14700,19700,156639,9040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-24,3012.429932,1007.419983,1185.500000,10.357149,4725.790039,15653.370117,1.50,72.847826,50,1665,...,3530,31950,6430,14600,4360,7890,30650,35200,45700,5130
2021-12-27,2999.550049,1011.359985,1185.920044,10.363698,4791.189941,15871.259766,1.48,72.847826,51,1661,...,3555,31850,6460,14850,4325,7890,30350,35200,45600,5220
2021-12-28,3020.239990,1027.439941,1185.650024,10.334326,4786.350098,15781.719727,1.49,72.847826,51,1675,...,3550,32200,6510,14650,4325,7890,30700,35000,45950,5270


In [20]:
total_stock.isna().sum().sum()

0

In [21]:
total_stock.to_csv('./data/kospi_financeIndex_and_stock.csv')